In [26]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os # need for file enumeration
import pickle # to save model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils

# For reproducibility
np.random.seed(42)

In [3]:
# get our file listings for train and test
train_files = os.listdir('images/train/')
test_files = os.listdir('images/test/')

In [5]:
# convert filenames into dataframes
train_df = pd.DataFrame(train_files, columns = ['filename'])
test_df = pd.DataFrame(test_files, columns = ['filename'])

In [15]:
# label 'em
train_df['hotdog'] = train_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')
test_df['hotdog'] = test_df['filename'].map(lambda x:
    'hotdog' if x.startswith('hotdog') else 'nothotdog')

In [16]:
# set up imagedatagenerators
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train_df,
        directory = 'images/train',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')
validation_generator = test_datagen.flow_from_dataframe(
        dataframe = test_df,
        directory = 'images/test',
        x_col = 'filename',
        y_col = 'hotdog',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary')

Found 3000 validated image filenames belonging to 2 classes.
Found 644 validated image filenames belonging to 2 classes.


In [24]:
# train baseline
train_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [25]:
# test baseline
test_df['hotdog'].value_counts(normalize = True)

hotdog       0.5
nothotdog    0.5
Name: hotdog, dtype: float64

In [18]:
# set up model
cnn_model = Sequential()

cnn_model.add(Conv2D( 
  filters = 6, # number of filters
  kernel_size = (3, 3), # height/width of filter
  activation = 'relu', # activation function 
  input_shape = (300, 300, 1) # shape of input (image)
))

cnn_model.add(MaxPooling2D(pool_size = (2, 2))) # dimensions of region of pooling

cnn_model.add(Conv2D(filters = 16,
                     kernel_size = (3, 3),
                     activation = 'relu'))

cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 128, activation = 'relu'))

cnn_model.add(Dense(units = 1, activation = 'sigmoid'))

cnn_model.compile(loss = 'binary_crossentropy',
                  optimizer ='adam',
                  metrics = ['accuracy'])


In [20]:
# fit model
cnn_model.fit(
        train_generator,
        epochs = 5,
        validation_data = validation_generator)

Epoch 1/5
94/94 [==============================] - 75s 797ms/step - loss: 0.6622 - accuracy: 0.5990 - val_loss: 0.6664 - val_accuracy: 0.5776
Epoch 2/5
94/94 [==============================] - 56s 599ms/step - loss: 0.6151 - accuracy: 0.6633 - val_loss: 0.7246 - val_accuracy: 0.6165
Epoch 3/5
94/94 [==============================] - 66s 706ms/step - loss: 0.5786 - accuracy: 0.6930 - val_loss: 0.6684 - val_accuracy: 0.6196
Epoch 4/5
94/94 [==============================] - 63s 670ms/step - loss: 0.5722 - accuracy: 0.7003 - val_loss: 0.6070 - val_accuracy: 0.6957
Epoch 5/5
94/94 [==============================] - 62s 653ms/step - loss: 0.5465 - accuracy: 0.7280 - val_loss: 0.6226 - val_accuracy: 0.6848


In [28]:
with open('./models/cnn_1.pkl', 'wb') as f:
    pickle.dump(cnn_model, f)

INFO:tensorflow:Assets written to: ram://04246d1e-8d85-440c-8a7a-38ba89780410/assets


INFO:tensorflow:Assets written to: ram://04246d1e-8d85-440c-8a7a-38ba89780410/assets
